<a href="https://colab.research.google.com/github/Abhi3030/BasicPython/blob/main/Poweroutput_prediction_Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.9 MB/s eta 0:00:00


In [2]:
# Import necessary libraries for data handling, modeling, evaluation, and UI
import streamlit as st           # For creating the interactive UI
import pandas as pd              # For data manipulation
import numpy as np               # For numerical operations
import matplotlib.pyplot as plt  # For plotting (if needed)
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# --------------------------------------------------------------------------
# Modeling Approach:
#
# - Task: Regression to predict net electrical energy output (PE) from ambient measurements.
# - Features: Temperature (T), Ambient Pressure (AP), Relative Humidity (RH), Exhaust Vacuum (V)
# - Algorithms: Using Linear Regression (baseline) and Random Forest Regression (to capture non-linearities)
#
# Data Splitting Strategy:
# - 70% Training, 10% Validation, and 20% Test.
#   1. First split 20% of the data as the test set.
#   2. Then, from the remaining 80%, allocate 12.5% for validation (0.125 * 80% = 10% overall)
##      and 87.5% for training (0.875 * 80% = 70% overall).

In [5]:
# Caching the data load function to avoid unnecessary reloads
@st.cache_data
def load_data():
    # Ensure the 'CCPP_data.csv' file is in the same directory or adjust the path accordingly
    data = pd.read_csv('/content/drive/MyDrive/Colab Dataset/CCPP_data.csv')
    return data

2025-02-15 22:29:35.306 No runtime found, using MemoryCacheStorageManager


In [6]:
# Train and select the best model using a separate validation set
@st.cache_resource
def train_model():
    # Load the dataset
    data = load_data()

    # Define features and target variable
    feature_cols = ['AT', 'AP', 'RH', 'V']  # Ambient measurements
    target_col = 'PE'                      # Net electrical energy output (MW)
    X = data[feature_cols]
    y = data[target_col]

    # First, split the data into 80% (training+validation) and 20% test set
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

    # Now split the 80% into training and validation sets:
    # 10% of the full data (i.e., 12.5% of 80%) for validation and the rest for training.
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, random_state=42)

    # -----------------------------
    # Model Building & Comparison:
    # -----------------------------
    # Model 1: Linear Regression
    lr_model = LinearRegression()
    lr_model.fit(X_train, y_train)
    lr_val_pred = lr_model.predict(X_val)
    lr_val_rmse = np.sqrt(mean_squared_error(y_val, lr_val_pred))

    # Model 2: Random Forest Regression with typical default hyperparameters
    rf_model = RandomForestRegressor(random_state=42, n_estimators=100, max_depth=10)
    rf_model.fit(X_train, y_train)
    rf_val_pred = rf_model.predict(X_val)
    rf_val_rmse = np.sqrt(mean_squared_error(y_val, rf_val_pred))

    # Select the model with the lower RMSE on the validation set
    if rf_val_rmse < lr_val_rmse:
        selected_model_name = "Random Forest Regression"
        best_model = rf_model
        val_rmse = rf_val_rmse
    else:
        selected_model_name = "Linear Regression"
        best_model = lr_model
        val_rmse = lr_val_rmse

    # Retrain the selected model on the combined training and validation sets for the final model
    X_train_final = pd.concat([X_train, X_val])
    y_train_final = pd.concat([y_train, y_val])
    best_model.fit(X_train_final, y_train_final)

    # -----------------------------
    # Final Model Evaluation:
    # -----------------------------
    # Evaluate the final model on the test set to get unbiased performance metrics
    y_test_pred = best_model.predict(X_test)
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    test_mae = mean_absolute_error(y_test, y_test_pred)
    test_r2 = r2_score(y_test, y_test_pred)

    # Pack the metrics in a dictionary to display in the UI
    metrics = {
        "Selected Model": selected_model_name,
        "Validation RMSE": val_rmse,
        "Test RMSE": test_rmse,
        "Test MAE": test_mae,
        "Test R²": test_r2
    }

    return best_model, metrics

# --------------------------------------------------------------------------
# Streamlit UI: User Input and Model Prediction
#
# - The UI allows users to input ambient environmental measurements.
# - The model predicts the net electrical energy output (PE) based on these inputs.
# - Model evaluation metrics are displayed for user interpretation.
# --------------------------------------------------------------------------
def main():
    st.title("Combined Cycle Power Plant Energy Output Prediction")
    st.write(
        "This application predicts the net hourly electrical energy output (PE) of a combined cycle power plant "
        "based on ambient environmental measurements."
    )

    # Sidebar: User inputs for feature values
    st.sidebar.header("Input Ambient Conditions")
    def user_input_features():
        AT = st.sidebar.number_input("Temperature (°C)", min_value=1.81, max_value=37.11, value=20.0)
        AP = st.sidebar.number_input("Ambient Pressure (milibar)", min_value=992.89, max_value=1033.30, value=1013.25)
        RH = st.sidebar.number_input("Relative Humidity (%)", min_value=25.56, max_value=100.16, value=50.0)
        V = st.sidebar.number_input("Exhaust Vacuum (cm Hg)", min_value=25.36, max_value=81.56, value=50.0)

        # Create a DataFrame for the input features to match the model's expected format
        input_data = {"AT": AT, "AP": AP, "RH": RH, "V": V}
        features = pd.DataFrame(input_data, index=[0])
        return features

    input_df = user_input_features()

    # Load and train the final model, then get the performance metrics
    final_model, metrics = train_model()

    # Display model evaluation metrics for interpretation
    st.subheader("Model Evaluation Metrics")
    st.write(metrics)

    # Use the trained model to predict based on user inputs
    prediction = final_model.predict(input_df)

    st.subheader("Prediction")
    st.write(f"Predicted net hourly electrical energy output (PE): {prediction[0]:.2f} MW")

    # Optionally display the user input features for confirmation
    st.subheader("User Input Features")
    st.write(input_df)

# Run the Streamlit application
if __name__ == '__main__':
    main()

2025-02-15 22:29:39.102 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-15 22:29:39.104 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-15 22:29:39.106 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-15 22:29:39.114 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-15 22:29:39.117 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-15 22:29:39.121 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-15 22:29:39.124 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-15 22:29:39.126 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [11]:
%%writefile app.py

Writing app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8000




⠙⠹⠸
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.106.30.70:8501

⠼⠴your url is: https://four-bobcats-cross.loca.lt


In [16]:
!curl https://loca.lt/mytunnelpassword

34.106.30.70